In [3]:
import os
import torch
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough

# 加载嵌入模型
embedding_model = HuggingFaceEmbeddings(
    model_name="/Users/zhangyf/llm/bge-base-zh-v1.5",
    model_kwargs={"device": "mps" if torch.mps.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# 初始化 Chroma 客户端
vectorstore = Chroma(
    persist_directory="vectorstore",
    embedding_function=embedding_model,
)

# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 检索方式，similarity 或 mmr
    search_kwargs={"k": 3},
)


In [7]:
# 检索与生成链条
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")
# 大模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)
# 本地模型


# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

问题：{query}

回答：""",
)



# RAG 链条
rag_chain = (
    {"context": retriever | format_docs, "query": RunnablePassthrough()}
    | prompt
    | (lambda x: print(x.text, end="") or x)
    | llm
    | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

# query = "不动产或者动产被占有人占有怎么办"
query = "今天天气怎么样？"
response = rag_chain.invoke(query)
print(response)



你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：(一 ) 政府 采购 情况 说 明

2023 年 是 深入 推 进 实施 “十 四 五 ?战略 规划 、 五 规划 目标 实现 的 关键 之 年 , 抢 抓 机 遇 争 取 承 担 国 推动 * 十 家 重大 科 四 技 任务 ， 管 理 运行 好 重大 观测 设施 ， 组 织 做 好 各 类 科学 数据 处 理 的 应 用 研究 工作 ， 努 力 产 出 更 多 更 大 科学 成 果 。

2023 年 工作 总 体 思路 是 : 深入 学 习 贯 彻 党 的 二 十 大 精 神 ， 沙 实 党 的 全 面 领导 ， 落 实 党 中 央 、 国 务 院 和 院 党 组 重 决策 部 署 ， 面 向 国家 重大 需求 、 面 向 国际 天 文科 技 前 沿 、 面 向 经 济 主 战场 ， 加 快 j 全 进 国 家 “十 四 五 ?科技 发 展 规划 实施 ， 加 快 建设 和 持续 推进 国家 重点 实验 室 重 组 工作 ， 加 快 协 同 凝 聚 天 文 领域 科研 队伍 ， 加 强 青年 人 才 队 伍 建设 ， 加 质量 发 展 新 阶段 ， 为 创新 型 国 ATK. 更 REN 家 和 世界 科技 强国 建设 做 出

问题：今天天气怎么样？

回答：我不知道


In [8]:
# 带历史对话记录
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")

# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "history", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

历史消息：[{history}]

问题：{query}

回答：""",
)

# 大模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)

# 历史消息
history = []

# 格式化历史消息
def format_history(history):
    # 只保留最近 3 轮对话记录
    max_epoch = 3
    if len(history) > 2 * max_epoch:
        history = history[-2 * max_epoch :]
    return "\n".join([f"{i['role']}：{i['content']}" for i in history])

# RAG 链条
rag_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["query"], k=3)),
        "history": lambda x: format_history(x["history"]),
        "query": lambda x: x["query"],
    }
    | prompt
    | (lambda x: print(x.text, end="") or x)
    | llm
    | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

query_list = ["不动产或者动产被人占有怎么办", "那要是被损毁了呢"]
for query in query_list:
    print(f"===== 查询: {query} =====")
    response = rag_chain.invoke({"query": query, "history": history})
    print(response, end="\n\n")
    history.extend(
        [
            {"role": "用户", "content": query},
            {"role": "助手", "content": response},
        ]
    )


===== 查询: 不动产或者动产被人占有怎么办 =====

你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：第四百五十七条　留置权人对留置财产丧失占有或者留置权人接受债务人另行提供担保的，留置权消灭。

第五分编　占有

第二十章　占有

第四百五十八条　基于合同关系等产生的占有，有关不动产或者动产的使用、收益、违约责任等，按照合同约定；合同没有约定或者约定不明确的，依照有关法律规定。

第四百五十九条　占有人因使用占有的不动产或者动产，致使该不动产或者动产受到损害的，恶意占有人应当承担赔偿责任。

第四百六十条　不动产或者动产被占有人占有的，权利人可以请求返还原物及其孳息；但是，应当支付善意占有人因维护该不动产或者动产支出的必要费用。

第四百六十一条　占有的不动产或者动产毁损、灭失，该不动产或者动产的权利人请求赔偿的，占有人应当将因毁损、灭失取得的保险金、赔偿金或者补偿金等返还给权利人；权利人的损害未得到足够弥补的，恶意占有人还应当赔偿损失。

第二百一十九条　利害关系人不得公开、非法使用权利人的不动产登记资料。

第二百二十条　权利人、利害关系人认为不动产登记簿记载的事项错误的，可以申请更正登记。不动产登记簿记载的权利人书面同意更正或者有证据证明登记确有错误的，登记机构应当予以更正。

不动产登记簿记载的权利人不同意更正的，利害关系人可以申请异议登记。登记机构予以异议登记，申请人自异议登记之日起十五日内不提起诉讼的，异议登记失效。异议登记不当，造成权利人损害的，权利人可以向申请人请求损害赔偿。

第二百二十一条　当事人签订买卖房屋的协议或者签订其他不动产物权的协议，为保障将来实现物权，按照约定可以向登记机构申请预告登记。预告登记后，未经预告登记的权利人同意，处分该不动产的，不发生物权效力。

预告登记后，债权消灭或者自能够进行不动产登记之日起九十日内未申请登记的，预告登记失效。

第二百二十二条　当事人提供虚假材料申请登记，造成他人损害的，应当承担赔偿责任。

因登记错误，造成他人损害的，登记机构应当承担赔偿责任。登记机构赔偿后，可以向造成登记错误的人追偿。

第二百二十三条　不动产登记费按件收取，不得按照不动产的面积、体积或者价款的比例收取。

第二